

This code is based on the work "Dynamic Point Fields: Learning Surface Deformation by **Author**: [Sergey Prokudin](https://scholar.google.com/citations?user=xSywCzAAAAAJ).
[[Project Page](https://sergeyprokudin.github.io/dpf/)]
[[Paper](https://arxiv.org/abs/2304.02626)]
[[Video](https://www.youtube.com/watch?v=i-9eAgS8HEA)]
[[GitHub](https://github.com/sergeyprokudin/dpf)]
 and was adapted to the set task of "Same Class Deformation". Where a source model will be deformed to represent a target model, with AIAP as a limiting factor to keep the source shape to some degree.

### Main steps:

1. Download the mesh of choice (optional)
2. Sample points from the surface to form initial point cloud;
3. Optimise point locations and normals to fit the target mesh and its renderings, w.r.t. Chamfer distance, normal Chamfer distance and image-based normal loss;
4. Render and save the final optimised cloud.
5. Define the SIREN deformation network.
6. Optimise deformation network based on the combination of losses: Chamfer distance, as-isometric-as-possible-regularisation (AIAP)
7. Visualise the deformed source mesh
8. Evaluate the deformed surface with its Chamfer distance


### Notes
For citing Dynamic Point Fields by Sergey Prokudin:
```bibtex
@article{prokudin2023dynamic,
                  title={Dynamic Point Fields},
                  author={Prokudin, Sergey and Ma, Qianli and Raafat, Maxime and Valentin, Julien and Tang, Siyu},
                  journal={arXiv preprint arXiv:2304.02626},
                  year={2023}
                }
```


In [13]:
# @title Install dependencies: pytorch3d, point-cloud-utils, trimesh

import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
"""
from zipfile import ZipFile
import os

!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip

with ZipFile('ModelNet10.zip', 'r') as ziped_file:
    ziped_file.extractall()
"""


#Easiest pair

source_file="/home/martin/Changed_Dynamic_Point_Field/ModelNet10/table/train/table_0025.off"
target_file="/home/martin/Changed_Dynamic_Point_Field/ModelNet10/table/train/table_0020.off"
test_file="/home/martin/Changed_Dynamic_Point_Field/ModelNet10/table/train/table_0031.off"

#another round table
#"ModelNet10\table\train\table_0057.off"

#"ModelNet10\table\train\table_0057.off"

#"/home/martin/Changed_Dynamic_Point_Field/ModelNet10/sofa/train/sofa_0045.off"
#"/home/martin/Changed_Dynamic_Point_Field/ModelNet10/sofa/train/sofa_0062.off"
#"/home/martin/Changed_Dynamic_Point_Field/ModelNet10/sofa/train/sofa_0042.off"

#"/home/martin/Changed_Dynamic_Point_Field/ModelNet10/chair/train/chair_0003.off"

#"/home/martin/Changed_Dynamic_Point_Field/Dino/mesh_clean1.ply"
#t"/home/martin/Changed_Dynamic_Point_Field/Dino/mesh_clean2.ply"
#"/home/martin/Changed_Dynamic_Point_Field/Dino/mesh_clean3.ply"


save_source='pcd_table0025.ply'
save_target='pcd_table0020.ply'
save_test='pcd_table0030.ply'

In [14]:
# @title Import libraries, define the necessary auxiliary functions (mesh normalisation, rendering, etc.)

import numpy as np

from tqdm import tqdm

#  load OBJ files as pytorch3D meshes.
from pytorch3d.io import load_objs_as_meshes
#from pytorch3d.io import _load_off_stream
# import class "meshes" - see https://pytorch3d.readthedocs.io/en/latest/modules/structures.html
from pytorch3d.structures import Meshes, Pointclouds
#from pytorch3d.off_io import MeshOffFormat

from pytorch3d.io import IO

# to load an off mesh--------------------------------------------
import trimesh
import matplotlib.pyplot as plt

# classes and modules from renderer - see https://pytorch3d.readthedocs.io/en/latest/modules/renderer/index.html
from pytorch3d.renderer import (
    Textures,
    #TextureVertex was added by me
    TexturesVertex,
    look_at_view_transform,
    # Field of View
    FoVOrthographicCameras,
    Materials,
    #mapping from 3d scene to 2d image (not rendering , here is just the mapping)
    RasterizationSettings,
    BlendParams,
    MeshRenderer,
    #Rasterizes meshes into images
    MeshRasterizer,
    AmbientLights,
    #color shaing ?
    HardPhongShader,
)

# Constants for magic numbers
DEVICE = 'cuda'
DEFAULT_IMAGE_SIZE = 512
NUM_IMAGES = 100
DEFAULT_SCALE_VAL = 1.0
DEFAULT_RANDOM_SEED = 13

def normals_to_rgb(normals: torch.Tensor) -> torch.Tensor:
    """
    Convert mesh normals to RGB color representation.

    Args:
        normals (torch.Tensor): Mesh normals.

    Returns:
        torch.Tensor: RGB colors based on the normals.
    """
    # value range is now [0,1] , befor it was [-1,1]
    return torch.abs(normals * 0.5 + 0.5)

def get_normals_as_textures(mesh: Meshes) -> Meshes:
    """
    Create textures from mesh normals.

    Args:
        mesh (Meshes): Input mesh.

    Returns:
        Meshes: New mesh with normals as textures.
    """
    #get normals as tensor from mesh object
    normals = mesh.verts_normals_packed()

    #change normals into range [0,1] , unsqueeze is needed for the "Textures or TexturesVertex" function
    #"Textures or TexturesVertex" function creates with the "normals_to_rgb" colored textures for the meshes (see images above)
    colors=normals_to_rgb(normals)
    textures = TexturesVertex(colors.unsqueeze(0))

    #takes the original (input) meseh vertices and faces and adds the color textures to the vertices and faces
    return Meshes(mesh.verts_packed().unsqueeze(0), mesh.faces_packed().unsqueeze(0), textures)



def normalize_verts(vertices: torch.Tensor, scale=None, center=None) -> tuple:
    """
    Normalize vertex positions of a mesh.

    Args:
        vertices (torch.Tensor): Vertex positions.
        scale (float, optional): Scaling factor. Defaults to None.
        center (torch.Tensor, optional): Center of the mesh. Defaults to None.

    Returns:
        tuple: Normalized vertex positions, scaling factor, and center.
    """

    # if center andscale known then centers the vertices around the origin and scales them
    if scale is not None and center is not None:
        vertices = vertices - center
        vertices *= scale
    else:
      #calculate max and min of mesh vertices
        v_max, _ = torch.max(vertices, dim=0)
        v_min, _ = torch.min(vertices, dim=0)
        #calculate center of mesh
        center = (v_max + v_min) / 2.
        #centering around origin
        vertices = vertices - center
        #calc max distance from vertex to origin (euklidean)
        max_dist = torch.sqrt(torch.max(torch.sum(vertices**2, dim=-1)))
        #scaling the max dist vertex to 1
        scale = (1. / max_dist)
        vertices *= scale

        #return vertices , the scale and the center of the mesh
    return vertices, scale, center

def normalize_mesh(mesh: Meshes, scale=None, center=None) -> tuple:
    """
    Normalize a mesh.

    Args:
        mesh (Meshes): Input mesh.
        scale (float, optional): Scaling factor. Defaults to None.
        center (torch.Tensor, optional): Center of the mesh. Defaults to None.

    Returns:
        tuple: Normalized mesh, scaling factor, and center.
    """
    # use function from before
    vertices, scale, center = normalize_verts(mesh.verts_packed(), scale, center)
    #updates the mesh with the normalized vertices to create a new mesh
    normalized_mesh = mesh.update_padded(vertices.unsqueeze(0))
    return normalized_mesh, scale, center

def create_mesh_renderer(cameras, image_size=DEFAULT_IMAGE_SIZE, device='cuda'):
    """
    Create a mesh renderer.

    Args:
        cameras (FoVOrthographicCameras): Camera setup.
        image_size (int, optional): Image size. Defaults to DEFAULT_IMAGE_SIZE.
        device (str, optional): Device for computation. Defaults to 'cuda'.

    Returns:
        MeshRenderer: Mesh renderer.
    """
    materials = Materials(
        device=device,
        #Black and no shine
        specular_color=[[0.0, 0.0, 0.0]],
        shininess=0.0
    )

    raster_settings = RasterizationSettings(
        image_size=image_size,
        blur_radius=0.0,
        faces_per_pixel=1,
        bin_size=None,
        cull_backfaces=True
    )
    # background color ?
    blend_params = BlendParams(background_color=(0, 0, 0))

    lights = AmbientLights(ambient_color=(1, 1, 1), device=device)

    renderer = MeshRenderer(
        rasterizer=MeshRasterizer(
            cameras=cameras,
            raster_settings=raster_settings
        ),

        shader=HardPhongShader(device=device,
                               cameras=cameras,
                               blend_params=blend_params,
                               lights=lights,
                               materials=materials)
    )

    return renderer

def render_mesh(mesh, dist=1000, elev=0 , azim=0 ,image_size=DEFAULT_IMAGE_SIZE, radius=0.01, scale_val=0.02,device=DEVICE):
    """
    Render a mesh.

    Args:
        mesh (Meshes): Input mesh.
        dist (float, optional): Distance from the camera. Defaults to 1.
        elev (float, optional): Elevation angle. Defaults to 0.
        azim (float, optional): Azimuth angle. Defaults to 0.
        image_size (int, optional): Image size. Defaults to DEFAULT_IMAGE_SIZE.
        radius (float, optional): Radius. Defaults to 0.01.
        scale_val (float, optional): Scaling value. Defaults to 1.0.
        device (str, optional): Device to use for rendering. Defaults to DEVICE.

    Returns:
        torch.Tensor: Rendered image.
    """
    #get Rotation and Translation matrix for camera position at the set values
    R, T = look_at_view_transform(dist=dist, elev=elev, azim=azim)
    # move camera to the set position and scale
    cam = FoVOrthographicCameras(R=R, T=T, scale_xyz=((scale_val, scale_val, scale_val),)).to(device)

    renderer = create_mesh_renderer(cam, image_size=image_size)
   
    img = renderer(mesh, cameras=cam)[0]

    return img

def generate_render_data(mesh,
                         n_images=NUM_IMAGES,
                         scale_val=DEFAULT_SCALE_VAL,
                         rand_seed=DEFAULT_RANDOM_SEED,
                         image_size=DEFAULT_IMAGE_SIZE,
                         device=DEVICE):
    """
    Generate rendering data.

    Args:
        mesh (Meshes): Input mesh.
        n_images (int, optional): Number of images to generate. Defaults to NUM_IMAGES.
        scale_val (float, optional): Scaling value. Defaults to DEFAULT_SCALE_VAL.
        rand_seed (int, optional): Random seed. Defaults to DEFAULT_RANDOM_SEED.
        image_size (int, optional): Image size. Defaults to DEFAULT_IMAGE_SIZE.
        device (str, optional): Device to use for rendering. Defaults to DEVICE.
    Returns:
        tuple: Azimuth angles, elevation angles, and rendered images.
    """

    np.random.seed(rand_seed)
    ygts = []
    azims, elevs = [], []
    # get random angles
    #tqdm is "loading bar"
    for ix in tqdm(range(0, n_images)):
        azims.append(np.random.choice(360))
        elevs.append(np.random.choice(180))

        #render the mesh with given angles
        ygts.append(render_mesh(mesh.to(device),
                                azim=azims[ix],
                                elev=elevs[ix],
                                scale_val=scale_val,
                                image_size=image_size,
                                ).detach().cpu()[:, :, 0:3])

    #reset random seed ?
    np.random.seed(None)

    return azims, elevs, ygts

    # with this the function for mesh preparations are complete



In [15]:
# Load and normalise GT mesh

"""
# load mesh
mesh = trimesh.load('table_0001.off')
v,f = torch.from_numpy(mesh.vertices),torch.from_numpy(mesh.faces)
mesh_gt = Meshes(verts=[v], faces=[f]).to(DEVICE)
mesh_gt=mesh_gt.float()
#GT_MESH_PATH = 'lego_gt.obj'
"""


mesh_gt_source = IO().load_mesh(source_file, device=DEVICE)
mesh_gt_source = get_normals_as_textures(mesh_gt_source).to(DEVICE)
mesh_gt_source_norm, vscale_source, vcenter_source = normalize_mesh(mesh_gt_source)

mesh_gt_target = IO().load_mesh(target_file, device=DEVICE)
mesh_gt_target = get_normals_as_textures(mesh_gt_target).to(DEVICE)
mesh_gt_target_norm, vscale_target, vcenter_target = normalize_mesh(mesh_gt_target)

mesh_gt_test = IO().load_mesh(test_file, device=DEVICE)
mesh_gt_test = get_normals_as_textures(mesh_gt_test).to(DEVICE)
mesh_gt_test_norm, vscale_test, vcenter_test = normalize_mesh(mesh_gt_test)

In [16]:
# Generate renders of the GT mesh (used for training with image-space normal loss)

N_TRAIN_IMAGES = 200
#N_TRAIN_IMAGES = 100

azims_train_source, elevs_train_source,  ygts_train_source = generate_render_data(mesh_gt_source_norm,
                                                        n_images=N_TRAIN_IMAGES,
                                                        scale_val=1.0,
                                                        rand_seed=19,
                                                        image_size=DEFAULT_IMAGE_SIZE)

azims_train_target, elevs_train_target,  ygts_train_target = generate_render_data(mesh_gt_target_norm,
                                                        n_images=N_TRAIN_IMAGES,
                                                        scale_val=1.0,
                                                        rand_seed=19,
                                                        image_size=DEFAULT_IMAGE_SIZE)

azims_train_test, elevs_train_test,  ygts_train_test = generate_render_data(mesh_gt_test_norm,
                                                        n_images=N_TRAIN_IMAGES,
                                                        scale_val=1.0,
                                                        rand_seed=19,
                                                        image_size=DEFAULT_IMAGE_SIZE)

100%|█████████████████████████████████████████████████████████████████████████████████| 200/200 [00:06<00:00, 33.28it/s]


In [17]:
"""
import matplotlib.pyplot as plt

# visualised GT mesh
plt.figure()
plt.title("GT mesh source")
plt.imshow(ygts_train_source[np.random.choice(N_TRAIN_IMAGES)])

plt.figure()
plt.title("GT mesh target")
plt.imshow(ygts_train_target[np.random.choice(N_TRAIN_IMAGES)])
"""

'\nimport matplotlib.pyplot as plt\n\n# visualised GT mesh\nplt.figure()\nplt.title("GT mesh source")\nplt.imshow(ygts_train_source[np.random.choice(N_TRAIN_IMAGES)])\n\nplt.figure()\nplt.title("GT mesh target")\nplt.imshow(ygts_train_target[np.random.choice(N_TRAIN_IMAGES)])\n'

In [18]:
# @title Define functions for point rendering and optimisation

from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.structures import Pointclouds
from pytorch3d.renderer import (
    PointsRasterizationSettings,
    PointsRasterizer,
    PointsRenderer,
    AlphaCompositor
)
from pytorch3d.loss import chamfer_distance

POINTS_PER_PIXEL = 50

def get_point_renderer(image_size, radius=0.05, points_per_pixel=50):

  raster_settings = PointsRasterizationSettings(
      image_size=image_size,
      radius = radius,
      points_per_pixel = points_per_pixel
      )

  rasterizer = PointsRasterizer(cameras=FoVOrthographicCameras(),
                                raster_settings=raster_settings)
  renderer = PointsRenderer(
      rasterizer=rasterizer,
      compositor=AlphaCompositor(background_color=(1, 1, 1))

  )

  return renderer

def render_points(x, xf, dist=1, elev=0, azim=0, image_size=512,
                  radius=0.01, points_per_pixel=50, scale_val=1.0,
                  device=DEVICE):

  x = x.to(device)
  xf = xf.to(device)
  renderer = get_point_renderer(image_size=image_size, radius=radius, points_per_pixel=points_per_pixel)
  R, T = look_at_view_transform(dist=dist, elev=elev, azim=azim)
  cam = FoVOrthographicCameras(R=R, T=T, scale_xyz=((scale_val, scale_val, scale_val),)).to(device)

  pcl = Pointclouds(points=x.unsqueeze(0), features=xf.unsqueeze(0)).to(device)

  img = renderer(pcl, cameras=cam)[0]

  return img


# @title Pymeshlab normal estimation

# https://github.com/cnr-isti-vclab/PyMeshLab/blob/main/docs/filter_list.rst
# https://pymeshlab.readthedocs.io/en/0.2/tutorials/get_mesh_values.html

import pymeshlab

def get_normals_pymeshlab(x,  k=10, smoothiter=0):

  tmp_mesh_path = 'tmp.obj'
  pcu.save_mesh_v(tmp_mesh_path, x)

  ms = pymeshlab.MeshSet()
  ms.load_new_mesh(tmp_mesh_path)
  ms.compute_normal_for_point_clouds(k=k, smoothiter=smoothiter)
  #ms.save_current_mesh(trg_path, save_vertex_normal=True)
  #ms.generate_surface_reconstruction_screened_poisson(depth=8)
  m = ms.current_mesh()

  x = m.vertex_matrix()
  n = m.vertex_normal_matrix()

  return x, n


In [19]:
# Sample initial points and corresponding normals

import point_cloud_utils as pcu

# uncomment the line for desired number of points and corresponding radius
# NUM_POINTS, POINT_RADIUS = 10**4, 0.025
# NUM_POINTS, POINT_RADIUS = 10**5, 0.009
NUM_POINTS, POINT_RADIUS = 3*10**5, 0.007
# NUM_POINTS, POINT_RADIUS = 10**6, 0.004

#return points and corresponding normals
v0_source, n0_source = sample_points_from_meshes(mesh_gt_source.to(DEVICE),
                                   return_normals=True,
                                   num_samples=NUM_POINTS)
# and saved as paramters for optimization, grad is for gradient descent for autograd?
v0_source = torch.nn.Parameter(v0_source[0].clone().to(DEVICE), requires_grad=True)
n0_source = torch.nn.Parameter(n0_source[0].clone().to(DEVICE), requires_grad=True)

#save as "ply" file
# save the initial cloud for evaluation at the end
pcu.save_mesh_vn('init_source.ply', v0_source.detach().cpu().numpy(), n0_source.detach().cpu().numpy())



v0_target, n0_target = sample_points_from_meshes(mesh_gt_target.to(DEVICE),
                                   return_normals=True,
                                   num_samples=NUM_POINTS)

# and saved as paramters for optimization
v0_target = torch.nn.Parameter(v0_target[0].clone().to(DEVICE), requires_grad=True)
n0_target = torch.nn.Parameter(n0_target[0].clone().to(DEVICE), requires_grad=True)

#save as "ply" file
pcu.save_mesh_vn('init_target.ply', v0_target.detach().cpu().numpy(), n0_target.detach().cpu().numpy())

v0_test, n0_test = sample_points_from_meshes(mesh_gt_test.to(DEVICE),
                                   return_normals=True,
                                   num_samples=NUM_POINTS)

# and saved as paramters for optimization
v0_test = torch.nn.Parameter(v0_test[0].clone().to(DEVICE), requires_grad=True)
n0_test = torch.nn.Parameter(n0_test[0].clone().to(DEVICE), requires_grad=True)

#save as "ply" file
pcu.save_mesh_vn('init_test.ply', v0_test.detach().cpu().numpy(), n0_test.detach().cpu().numpy())

In [20]:
# render initial, non-optimised cloud

#select random iamge from train data set
DEMO_FRAME_IX = np.random.choice(N_TRAIN_IMAGES)

#normalize the points of the pointcloud v0 around the origin, like before with mesehs
v0_norm_source, _, _ = normalize_verts(v0_source, vscale_source, vcenter_source)

ypred_init_source = render_points(v0_norm_source.to(DEVICE),
                      normals_to_rgb(n0_source.to(DEVICE)),
                      elev=elevs_train_source[DEMO_FRAME_IX],
                      azim=azims_train_source[DEMO_FRAME_IX],
                      image_size=DEFAULT_IMAGE_SIZE,
                      points_per_pixel=POINTS_PER_PIXEL,
                      radius=POINT_RADIUS).detach().cpu().numpy()

#print a demo of the point cloud and the corresponding ground truth mesh
ygt_demo_source = ygts_train_source[DEMO_FRAME_IX].detach().cpu().numpy()
"""
plt.figure()
plt.title("Initial cloud (non-optimized) source")
plt.imshow(ypred_init_source)

plt.figure()
plt.title("Ground truth mesh source")
plt.imshow(ygt_demo_source)
"""

#select random iamge from train data set
DEMO_FRAME_IX = np.random.choice(N_TRAIN_IMAGES)

#normalize the points(vectrices) of the pointcloud v0 around the origin, like before with mesehs
v0_norm_target, _, _ = normalize_verts(v0_target, vscale_target, vcenter_target)

ypred_init_target = render_points(v0_norm_target.to(DEVICE),
                      normals_to_rgb(n0_target.to(DEVICE)),
                      elev=elevs_train_target[DEMO_FRAME_IX],
                      azim=azims_train_target[DEMO_FRAME_IX],
                      image_size=DEFAULT_IMAGE_SIZE,
                      points_per_pixel=POINTS_PER_PIXEL,
                      radius=POINT_RADIUS).detach().cpu().numpy()

#print a demo of the point cloud and the corresponding ground truth mesh
ygt_demo_test = ygts_train_test[DEMO_FRAME_IX].detach().cpu().numpy()


#select random iamge from train data set
DEMO_FRAME_IX = np.random.choice(N_TRAIN_IMAGES)

#normalize the points(vectrices) of the pointcloud v0 around the origin, like before with mesehs
v0_norm_test, _, _ = normalize_verts(v0_test, vscale_test, vcenter_test)

ypred_init_test = render_points(v0_norm_test.to(DEVICE),
                      normals_to_rgb(n0_test.to(DEVICE)),
                      elev=elevs_train_test[DEMO_FRAME_IX],
                      azim=azims_train_test[DEMO_FRAME_IX],
                      image_size=DEFAULT_IMAGE_SIZE,
                      points_per_pixel=POINTS_PER_PIXEL,
                      radius=POINT_RADIUS).detach().cpu().numpy()

#print a demo of the point cloud and the corresponding ground truth mesh
ygt_demo_test = ygts_train_test[DEMO_FRAME_IX].detach().cpu().numpy()

"""
plt.figure()
plt.title("Initial cloud (non-optimized) target")
plt.imshow(ypred_init_target)

plt.figure()
plt.title("Ground truth mesh target")
plt.imshow(ygt_demo_target)
"""

'\nplt.figure()\nplt.title("Initial cloud (non-optimized) target")\nplt.imshow(ypred_init_target)\n\nplt.figure()\nplt.title("Ground truth mesh target")\nplt.imshow(ygt_demo_target)\n'

In [21]:
# @title Run point cloud optimisation
#NUM_EPOCHS = 4
#NUM_GT_EVAL_POINTS = 10**6
#N_ITERS_PER_EPOCH = 20

#NUM_EPOCHS = 1
#NUM_GT_EVAL_POINTS = 10**2
#N_ITERS_PER_EPOCH = 20

NUM_EPOCHS = 4
NUM_GT_EVAL_POINTS = 10**5
N_ITERS_PER_EPOCH = 20


#vs_source = []
#ns_source = []

#mean squared error loss function
mse_loss = torch.nn.MSELoss()

#Adam = Adaptive Moment Estimation (AdaGrad and RMSProp)
#optimazation used to train deep learning models
optv = torch.optim.Adam([v0_source], lr=1.0e-4)
optn = torch.optim.Adam([n0_source], lr=1.0e-2)

#scheduler for vertex , when a there is only little change in loss then the learning rate is reduced
schedv = torch.optim.lr_scheduler.ReduceLROnPlateau(optv, patience=1)


#same for normals
schedn = torch.optim.lr_scheduler.ReduceLROnPlateau(optn, patience=1)


#total loss
tloss = 0
n_r = 0

#loading bar
for i in tqdm(range(0, NUM_EPOCHS)):

  # random set of training images is selected ( wiht no replace)
  epoch_fids = np.random.choice(len(azims_train_source), N_ITERS_PER_EPOCH, replace=False)


  chamfer_loss_total = 0
  l2_norm_total = 0
  cosine_normal_loss_total = 0
  n_r = 0
# work through the selected set
  for ix in epoch_fids:

#sample points from mesh
    vgt_source, vgtn_source = sample_points_from_meshes(mesh_gt_source.to(DEVICE),
                        return_normals=True,
                        num_samples=NUM_GT_EVAL_POINTS)

#normalize the points
    v0_norm_source, _, _ = normalize_verts(v0_source, vscale_source, vcenter_source)
    ypred_source = render_points(v0_norm_source.to(DEVICE),
                      normals_to_rgb(n0_source.to(DEVICE)),
                      elev=elevs_train_source[ix],
                      azim=azims_train_source[ix],
                      image_size=DEFAULT_IMAGE_SIZE,
                      points_per_pixel=POINTS_PER_PIXEL,
                      radius=POINT_RADIUS)

#mse error between the ground truth mesh and the render sampled points
    l2_norm = 10*mse_loss(ygts_train_source[ix].to(DEVICE), ypred_source)
#chamfer loss - the sum of the min distance between each point of cloud A to B and cloud B to A
#cosine loss - how similar are the normals of these closest points
    chamfer_loss, cosine_normal_loss = chamfer_distance(vgt_source, v0_source.unsqueeze(0), x_normals=vgtn_source, y_normals=n0_source.unsqueeze(0), norm=2)

    chamfer_loss, normal_loss = 10**4*chamfer_loss, cosine_normal_loss
    loss = chamfer_loss + cosine_normal_loss + 10*l2_norm

    chamfer_loss_total += float(chamfer_loss)
    cosine_normal_loss_total += float(cosine_normal_loss)
    l2_norm_total += float(l2_norm)
    n_r += 1
    optv.zero_grad()
    optn.zero_grad()
    loss.backward()
    optv.step()
    optn.step()


  chamfer_loss_total /= n_r
  l2_norm_total /= n_r
  cosine_normal_loss_total /= n_r
  print("Chamfer loss: %f, cosine normal loss: %f,  L2 image normal loss: %f" % (chamfer_loss_total, cosine_normal_loss_total, l2_norm_total))
  total_loss = chamfer_loss_total + l2_norm_total
  schedv.step(float(total_loss))
  schedn.step(float(total_loss))

 25%|█████████████████████▎                                                               | 1/4 [00:08<00:25,  8.36s/it]

Chamfer loss: 68.252916, cosine normal loss: 0.025553,  L2 image normal loss: 7.183423


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:16<00:16,  8.33s/it]

Chamfer loss: 67.961147, cosine normal loss: 0.025488,  L2 image normal loss: 7.157850


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:24<00:08,  8.31s/it]

Chamfer loss: 67.773327, cosine normal loss: 0.026230,  L2 image normal loss: 7.108978


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:33<00:00,  8.33s/it]

Chamfer loss: 67.560881, cosine normal loss: 0.027013,  L2 image normal loss: 6.966903


In [22]:
## @title Run point cloud optimisation


#mean squared error loss function
mse_loss = torch.nn.MSELoss()

#Adam = Adaptive Moment Estimation (AdaGrad and RMSProp)
#optimazation used to train deep learning models
optv = torch.optim.Adam([v0_target], lr=1.0e-4)
optn = torch.optim.Adam([n0_target], lr=1.0e-2)

#scheduler for vertex , when a there is only little change in loss then the learning rate is reduced
schedv = torch.optim.lr_scheduler.ReduceLROnPlateau(optv, patience=1)

#same for normals
schedn = torch.optim.lr_scheduler.ReduceLROnPlateau(optn, patience=1)

tloss = 0
n_r = 0

#loading bar
for i in tqdm(range(0, NUM_EPOCHS)):

  # random set of training images is selected ( wiht no replace)
  epoch_fids = np.random.choice(len(azims_train_target), N_ITERS_PER_EPOCH, replace=False)


  chamfer_loss_total = 0
  l2_norm_total = 0
  cosine_normal_loss_total = 0
  n_r = 0
# work through the selected set
  for ix in epoch_fids:

    #sample points from mesh
    vgt_target, vgtn_target = sample_points_from_meshes(mesh_gt_target.to(DEVICE),
                        return_normals=True,
                        num_samples=NUM_GT_EVAL_POINTS)

    #normalize the points
    v0_norm_target, _, _ = normalize_verts(v0_target, vscale_target, vcenter_target)
    ypred_target = render_points(v0_norm_target.to(DEVICE),
                      normals_to_rgb(n0_target.to(DEVICE)),
                      elev=elevs_train_target[ix],
                      azim=azims_train_target[ix],
                      image_size=DEFAULT_IMAGE_SIZE,
                      points_per_pixel=POINTS_PER_PIXEL,
                      radius=POINT_RADIUS)

    #mse error between the ground truth mesh and the render sampled points (why times 10???)
    l2_norm = 10*mse_loss(ygts_train_target[ix].to(DEVICE), ypred_target)
    #chamfer loss - the sum of the min distance between each point of cloud A to B and cloud B to A
    #cosine loss - how similar are the normals of these closes points ?
    chamfer_loss, cosine_normal_loss = chamfer_distance(vgt_target, v0_target.unsqueeze(0), x_normals=vgtn_target, y_normals=n0_target.unsqueeze(0), norm=2)

    chamfer_loss, normal_loss = 10**4*chamfer_loss, cosine_normal_loss
    loss = chamfer_loss + cosine_normal_loss + 10*l2_norm

    chamfer_loss_total += float(chamfer_loss)
    cosine_normal_loss_total += float(cosine_normal_loss)
    l2_norm_total += float(l2_norm)
    n_r += 1
    optv.zero_grad()
    optn.zero_grad()
    loss.backward()
    optv.step()
    optn.step()

    # n_r number of iterations ????
  chamfer_loss_total /= n_r
  l2_norm_total /= n_r
  cosine_normal_loss_total /= n_r
  print("Chamfer loss: %f, cosine normal loss: %f,  L2 image normal loss: %f" % (chamfer_loss_total, cosine_normal_loss_total, l2_norm_total))
  total_loss = chamfer_loss_total + l2_norm_total
  schedv.step(float(total_loss))
  schedn.step(float(total_loss))

    

 25%|█████████████████████▎                                                               | 1/4 [00:08<00:24,  8.33s/it]

Chamfer loss: 411.876912, cosine normal loss: 0.026492,  L2 image normal loss: 7.968194


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:16<00:16,  8.33s/it]

Chamfer loss: 411.222572, cosine normal loss: 0.025679,  L2 image normal loss: 7.943242


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:24<00:08,  8.32s/it]

Chamfer loss: 410.630023, cosine normal loss: 0.025696,  L2 image normal loss: 7.593109


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:33<00:00,  8.32s/it]

Chamfer loss: 410.280080, cosine normal loss: 0.025926,  L2 image normal loss: 7.527642


In [23]:
## @title Run point cloud optimisation


#mean squared error loss function
mse_loss = torch.nn.MSELoss()

#Adam = Adaptive Moment Estimation (AdaGrad and RMSProp)
#optimazation used to train deep learning models
optv = torch.optim.Adam([v0_test], lr=1.0e-4)
optn = torch.optim.Adam([n0_test], lr=1.0e-2)

#scheduler for vertex , when a there is only little change in loss then the learning rate is reduced
schedv = torch.optim.lr_scheduler.ReduceLROnPlateau(optv, patience=1)


#same for normals
schedn = torch.optim.lr_scheduler.ReduceLROnPlateau(optn, patience=1)


tloss = 0
n_r = 0

#loading bar
for i in tqdm(range(0, NUM_EPOCHS)):

  # random set of training images is selected ( wiht no replace)
  epoch_fids = np.random.choice(len(azims_train_test), N_ITERS_PER_EPOCH, replace=False)


  chamfer_loss_total = 0
  l2_norm_total = 0
  cosine_normal_loss_total = 0
  n_r = 0
# work through the selected set
  for ix in epoch_fids:

#sample points from mesh
    vgt_test, vgtn_test = sample_points_from_meshes(mesh_gt_test.to(DEVICE),
                        return_normals=True,
                        num_samples=NUM_GT_EVAL_POINTS)

#normalize the points
    v0_norm_test, _, _ = normalize_verts(v0_test, vscale_test, vcenter_test)
    ypred_test = render_points(v0_norm_test.to(DEVICE),
                      normals_to_rgb(n0_test.to(DEVICE)),
                      elev=elevs_train_test[ix],
                      azim=azims_train_test[ix],
                      image_size=DEFAULT_IMAGE_SIZE,
                      points_per_pixel=POINTS_PER_PIXEL,
                      radius=POINT_RADIUS)

    #mse error between the ground truth mesh and the render sampled points (why times 10???)
    l2_norm = 10*mse_loss(ygts_train_test[ix].to(DEVICE), ypred_test)
    #chamfer loss - the sum of the min distance between each point of cloud A to B and cloud B to A
    #cosine loss - how similar are the normals of these closes points ?
    chamfer_loss, cosine_normal_loss = chamfer_distance(vgt_test, v0_test.unsqueeze(0), x_normals=vgtn_test, y_normals=n0_test.unsqueeze(0), norm=2)
      
    chamfer_loss, normal_loss = 10**4*chamfer_loss, cosine_normal_loss
    loss = chamfer_loss + cosine_normal_loss + 10*l2_norm

    chamfer_loss_total += float(chamfer_loss)
    cosine_normal_loss_total += float(cosine_normal_loss)
    l2_norm_total += float(l2_norm)
    n_r += 1
    optv.zero_grad()
    optn.zero_grad()
    loss.backward()
    optv.step()
    optn.step()


  chamfer_loss_total /= n_r
  l2_norm_total /= n_r
  cosine_normal_loss_total /= n_r
  print("Chamfer loss: %f, cosine normal loss: %f,  L2 image normal loss: %f" % (chamfer_loss_total, cosine_normal_loss_total, l2_norm_total))
  total_loss = chamfer_loss_total + l2_norm_total
  schedv.step(float(total_loss))
  schedn.step(float(total_loss))

 25%|█████████████████████▎                                                               | 1/4 [00:08<00:24,  8.32s/it]

Chamfer loss: 334.796335, cosine normal loss: 0.045757,  L2 image normal loss: 7.862214


 50%|██████████████████████████████████████████▌                                          | 2/4 [00:16<00:16,  8.33s/it]

Chamfer loss: 333.818434, cosine normal loss: 0.043866,  L2 image normal loss: 7.691204


 75%|███████████████████████████████████████████████████████████████▊                     | 3/4 [00:24<00:08,  8.32s/it]

Chamfer loss: 333.438287, cosine normal loss: 0.043066,  L2 image normal loss: 8.017503


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:33<00:00,  8.31s/it]

Chamfer loss: 332.972731, cosine normal loss: 0.043050,  L2 image normal loss: 7.609788


In [28]:
# Render the optimised cloud

ypred_source = render_points(v0_norm_source.to(DEVICE),
                      normals_to_rgb(n0_source.to(DEVICE)),
                      elev=elevs_train_source[DEMO_FRAME_IX],
                      azim=azims_train_source[DEMO_FRAME_IX],
                      image_size=DEFAULT_IMAGE_SIZE,
                      points_per_pixel=POINTS_PER_PIXEL,
                      radius=POINT_RADIUS)

ypred_source, npred_source = get_normals_pymeshlab(ypred_source.detach().cpu().numpy(), k=20, smoothiter=0)

plt.figure()
plt.title("Initial cloud (non-optimized)_source")
plt.imshow(ypred_init_source)

#ypred_source_color=normalize_verts_color(ypred_source)

plt.figure()
plt.title("Optimised point cloud_source")
plt.imshow(ypred_source.detach().cpu().numpy())

plt.figure()
plt.title("GT mesh_source")
plt.imshow(ygts_train_source[DEMO_FRAME_IX][:, :, 0:3].detach().cpu().numpy())

ValueError: Invalid number of dimensions for argument 'v_positions'. Expected 2 but got 3.

In [ ]:
## Render the optimised cloud

ypred_target = render_points(v0_norm_target.to(DEVICE),
                      normals_to_rgb(n0_target.to(DEVICE)),
                      elev=elevs_train_target[DEMO_FRAME_IX],
                      azim=azims_train_target[DEMO_FRAME_IX],
                      image_size=DEFAULT_IMAGE_SIZE,
                      points_per_pixel=POINTS_PER_PIXEL,
                      radius=POINT_RADIUS)

ypred_target, npred_target = get_normals_pymeshlab(ypred_target.cpu().numpy(), k=20, smoothiter=0)

plt.figure()
plt.title("Initial cloud (non-optimized)")
plt.imshow(ypred_init_target)

plt.figure()
plt.title("Optimised point cloud")
plt.imshow(ypred_target.detach().cpu().numpy())

plt.figure()
plt.title("GT mesh")
plt.imshow(ygts_train_target[DEMO_FRAME_IX][:, :, 0:3].detach().cpu().numpy())

In [ ]:
## Render the optimised cloud

ypred_test = render_points(v0_norm_test.to(DEVICE),
                      normals_to_rgb(n0_test.to(DEVICE)),
                      elev=elevs_train_test[DEMO_FRAME_IX],
                      azim=azims_train_test[DEMO_FRAME_IX],
                      image_size=DEFAULT_IMAGE_SIZE,
                      points_per_pixel=POINTS_PER_PIXEL,
                      radius=POINT_RADIUS)


plt.figure()
plt.title("Initial cloud (non-optimized)")
plt.imshow(ypred_init_test)

plt.figure()
plt.title("Optimised point cloud")
plt.imshow(ypred_test.detach().cpu().numpy())

plt.figure()
plt.title("GT mesh")
plt.imshow(ygts_train_test[DEMO_FRAME_IX][:, :, 0:3].detach().cpu().numpy())

In [ ]:
# Save the optimised cloud, evaluate
#pcu.save_mesh_vn('pcd_source.ply', v0_source.detach().cpu().numpy(), n0_source.detach().cpu().numpy())

pcu.save_mesh_vn(save_source, vpred__source.detach().cpu().numpy(),npred__source.detach().cpu().numpy())

#pcu.save_mesh_vn('pcd_target.ply', v0_norm_target.detach().cpu().numpy(), n0_target.detach().cpu().numpy())
pcu.save_mesh_vn(save_target, vpred__target.detach().cpu().numpy(),nred__target.detach().cpu().numpy())

#pcu.save_mesh_vn('pcd_target.ply', v0_norm_target.detach().cpu().numpy(), n0_target.detach().cpu().numpy())
pcu.save_mesh_vn(save_test, vpred__test.detach().cpu().numpy(),npred__test.detach().cpu().numpy())

print("-----------------------END--------------------------")

# Manually find points for the "forced" correspondence

In [ ]:
def find_ground_points(point_cloud):
    #z_threshold=0.00001
    point_num=30
    
    point_cloud = point_cloud.cpu().detach().numpy()

    # Find indices of points with lowest z values
    indices_z = np.argsort(point_cloud[:, 2])[:1000]

    # Find indices of points with lowest x values among the lowest z values
    z_points = point_cloud[indices_z]
    indices_xz = indices_z[np.argsort(z_points[:, 0])[:1000]]

    # Find indices of points with lowest values among the lowest zx values
    xz_points = point_cloud[indices_xz]
    indices_xyz = indices_xz[np.argsort(xz_points[:, 1])[:point_num]]
    print((f"xyz values id: {indices_xyz}"))
    print("Points with xyz values:")
    for idx in indices_xyz:
        print(point_cloud[idx])
        
    # Find indices of points with highest y values among the lowest zx values
    indices_xYz = indices_xz[np.argsort(-xz_points[:, 1])[:point_num]]
    print((f"xYz values id: {indices_xYz}"))
    print("Points with xYz values:")
    for idx in indices_xYz:
        print(point_cloud[idx])

    # Find indices of points with highestx values among the lowest z values
    indices_Xz = indices_z[np.argsort(-z_points[:, 0])[:1000]]

    # Find indices of points with lowest values among the lowest zx values
    Xz_points = point_cloud[indices_Xz]
    indices_Xyz = indices_Xz[np.argsort(Xz_points[:, 1])[:point_num]]
    print((f"Xyz values id: {indices_Xyz}"))
    print("Points with Xyz values:")
    for idx in indices_Xyz:
        print(point_cloud[idx])
        
    # Find indices of points with highest y values among the lowest zx values
    indices_XYz = indices_Xz[np.argsort(-Xz_points[:, 1])[:point_num]]
    print((f"XYz values id: {indices_XYz}"))
    print("Points with XYz values:")
    for idx in indices_XYz:
        print(point_cloud[idx])
    
    return 

def find_surface_points(point_cloud):
    point_num=40

    point_cloud = point_cloud.cpu().detach().numpy()

    # Find indices of points with highest z values
    indices_Z = np.argsort(-point_cloud[:, 2])[:1000]

    # Find indices of points with lowest x values among the highest z values
    Z_points = point_cloud[indices_Z]
    indices_xZ = indices_Z[np.argsort(Z_points[:, 0])[:1000]]

    # Find indices of points with lowest y values among the lowest x & highest z values
    xZ_points = point_cloud[indices_xZ]
    indices_xyZ = indices_xZ[np.argsort(xZ_points[:, 1])[:point_num]]
    print((f"xyZ values id: {indices_xyZ}"))
    print("Points with xyZ values:")
    for idx in indices_xyZ:
        print(point_cloud[idx])
        
    # Find indices of points with highest y values among the lowest x & highest z values
    indices_xYZ = indices_xZ[np.argsort(-xZ_points[:, 1])[:point_num]]
    print((f"xYz values id: {indices_xYZ}"))
    print("Points with xYz values:")
    for idx in indices_xYZ:
        print(point_cloud[idx])

    # Find indices of points with highest x values among the highestz values
    indices_XZ = indices_Z[np.argsort(-Z_points[:, 0])[:1000]]

    # Find indices of points with lowest values among the lowest zx values
    XZ_points = point_cloud[indices_XZ]
    indices_XyZ = indices_XZ[np.argsort(XZ_points[:, 1])[:point_num]]
    print((f"XyZ values id: {indices_XyZ}"))
    print("Points with Xyz values:")
    for idx in indices_XyZ:
        print(point_cloud[idx])
        
    # Find indices of points with highest y values among the lowest zx values
    indices_XYZ = indices_XZ[np.argsort(-XZ_points[:, 1])[:point_num]]
    print((f"XYZ values id: {indices_XYZ}"))
    print("Points with XYZ values:")
    for idx in indices_XYZ:
        print(point_cloud[idx])
    
    return 

    
    

#Nearest neighbor of these two pointcloud keypoints
def match_keypoints(src_kpoints, trg_kpoints):
    
    neighbors = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(trg_kpoints)
    distances, indices = neighbors.kneighbors(src_kpoints)
    return indices.flatten()



def normalize_verts_color(vertices: torch.Tensor, scale=None, center=None) -> tuple:
    """
    Normalize vertex positions of a mesh.

    Args:
        vertices (torch.Tensor): Vertex positions.
        scale (float, optional): Scaling factor. Defaults to None.
        center (torch.Tensor, optional): Center of the mesh. Defaults to None.

    Returns:
        tuple: Normalized vertex positions with corresponding "position colors", scaling factor, and center.
    """

    # if center andscale known then centers the vertices around the origin and scales them
    if scale is not None and center is not None:
        vertices = vertices - center
        vertices *= scale
    else:
        #calculate max and min of mesh vertices
        v_max, _ = torch.max(vertices, dim=0)
        v_min, _ = torch.min(vertices, dim=0)
        #calculate center of mesh
        center = (v_max + v_min) / 2.
        #centering around origin
        vertices = vertices - center
        #calc max distance from vertex to origin (euklidean)
        max_dist = torch.sqrt(torch.max(torch.sum(vertices**2, dim=-1)))
        #scaling the max dist vertex to 1
        scale = (1. / max_dist)
        vertices *= scale

        #now take the coordinates to change these values into color, depending on the position of the scaled version
        colors = torch.zeros_like(vertices)
        
        colors[:, 0] = vertices[:, 0]  # red for x
        colors[:, 1] = vertices[:, 1]  # green for y
        colors[:, 2] = vertices[:, 2]  #blue for z

        # normalization of color like for the vertices
        #calculate max and min of color
        max_values = colors.max(dim=0)[0]
        min_values = colors.min(dim=0)[0]
        
        # Normalize the color channels like above
        normalized_colors = (colors - min_values) / (max_values - min_values)

        #add the color tensor to vertex tensor
        vertices_with_colors = torch.cat((vertices, normalized_colors), dim=1)

        #return vertices with color , the scale and the center of the mesh
    return vertices_with_colors, scale, center

In [ ]:
############### with keypoints ###########################################################################
import open3d as o3d

def highlight_points(points,indices):

    points=points.cpu().detach().numpy()
    #indices=indices.long().cpu().numpy()
    indices = np.array(indices)
    geometries = o3d.geometry.TriangleMesh()

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    #spheres = []
    geometries = [pcd]
    for index in indices:
        #print(indices)
        #print(index)
        point_coord = points[index]
        
        print(points[index])
        #point_coord = np.reshape(point_coord, (3,))
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.01, resolution=20) #create a small sphere to represent point
        #sphere.translate(np.array([point_coord]))
        sphere.translate(point_coord) #translate this sphere to point

        #point_coord = np.reshape(point_coord, (3,))
        #sphere.translate(point_coord.reshape(-1))

        sphere.paint_uniform_color([0.81, 0.43, 0.79])  # Color the sphere
        geometries.append(sphere)
        #geometries.append(sphere)
    #geometries.paint_uniform_color([1.0, 0.0, 0.0])
    return geometries

In [ ]:
test_file='pcd_table0031.ply'

mesh_gt_test = torch.Tensor(pcu.load_mesh_v(test_file))

vtest_norm_colors,_,_=normalize_verts_color(torch.tensor(mesh_gt_test).to(DEVICE))
vtest_norm_colors=vtest_norm_colors.detach().clone()
vtest_pos = vtest_norm_colors[:, :3]
vtest_col = vtest_norm_colors[:, 3:]

find_ground_points(vtest_pos)


In [ ]:
print("##################################################################################################")
find_surface_points(vtest_pos)


In [ ]:
#for table20
ground_points=[263290,133196,268633,243997 ]
# for table 25
#ground_points=[69152,174507,268407,41659]
# ground points for table 31
ground_points=[168458,36783,257424,172839]

highlighted_points=highlight_points(vtest_pos,ground_points)
o3d.visualization.draw_geometries(highlighted_points)

In [ ]:
#for table 20
#surface_points=[77546,12920,118464,27233]
#surfacepoints for table25
#surface_points=[46472,257155,298507,205242]
#surfacepoints for table31
surface_points=[254413,169476,158686,38907]
highlighted_points=highlight_points(vtest_pos,surface_points)
o3d.visualization.draw_geometries(highlighted_points)